<a href="https://colab.research.google.com/github/geoskimoto/SNOTEL_ErrorAndOutlier_Tools/blob/main/errorFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Library Imports

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV, HuberRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
from scipy.stats import zscore 
# import statsmodels.api as sm
from functools import reduce

import requests
import xml.dom.minidom as minidom
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import datetime


General format of "Error Finder Object":


*   outlierDetection
  *   tempOutliers

*   logicTests
  *   SWEoutpacePREC
  *   Filters
  





# Helper Functions

In [ ]:
# Web Call to Access and Download Data of a Single Station from AWDB Web Service (SOAP API)

def SOAP_Call(stationtriplets, elementCD, begindate, enddate):

  # Create a dictionaries to store the data
  headers = {'Content-type': 'text/soap'}
  # current_dictionary = {}
  
  # Define Web Service URL
  URL = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"

  # Define Parameters for SOAP Elements (getData:current and getCentralTendencyData:normals)
  SOAP_current = '''
  <?xml version="1.0" encoding="UTF-8"?>
  <SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:q0="http://www.wcc.nrcs.usda.gov/ns/awdbWebService" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <SOAP-ENV:Body>
      <q0:getData>
        <stationTriplets>STATIONTRIPLETS</stationTriplets>
        <elementCd>ELEMENTCD</elementCd>   
        <ordinal>1</ordinal>
        <duration>DAILY</duration>
        <getFlags>false</getFlags>
        <beginDate>BEGINDATE</beginDate>
        <endDate>ENDDATE</endDate>
        <alwaysReturnDailyFeb29>false</alwaysReturnDailyFeb29>   
      </q0:getData>
    </SOAP-ENV:Body>
  </SOAP-ENV:Envelope>

  '''.strip()
  #Read GetData documents - If <alwaysReturnDailyFeb29> is set to true, will set a null for every non leap year on the 29th,  
  #which breaks this request when selecting date ranges that include Feb 29.
  #Possible element codes: PREC, WTEQ (Water Equivalent/SWE)
  

  # Post SOAP Elements to AWDB Web Service and process results - getData
  SOAP_current = SOAP_current.replace("ELEMENTCD", elementCD)
  SOAP_current = SOAP_current.replace("STATIONTRIPLETS", stationtriplets)
  SOAP_current = SOAP_current.replace("BEGINDATE", begindate)
  SOAP_current = SOAP_current.replace("ENDDATE", enddate)

  response_current = requests.post(URL, data=SOAP_current, headers=headers)
  xmldoc = minidom.parseString(response_current.text)

  val_length = len(xmldoc.getElementsByTagName('values'))
  data = pd.DataFrame([xmldoc.getElementsByTagName('values')[i].firstChild.data for i in range(0,val_length)])

  date = datetime.datetime.strptime(begindate, "%m/%d/%Y").date()  #https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior; .date() after .strptime just tells it to make it a datetime object.  Def necessary.
  # print(date)
  Date = []                                                       
  for i in range(0, val_length): 
    date += datetime.timedelta(days=1)
    Date.append((date))

  # {str(xmldoc.getElementsByTagName('stationTriplet')[0].firstChild.data):{Date[j]:xmldoc.getElementsByTagName('values')[j].firstChild.data} for j in range(3)}

  data['Date'] = Date
  data.columns = [f'{elementCD}','Date']
  data.set_index('Date', inplace=True)
  
  data[f'{elementCD}'] = list(map(lambda x: float(x), data[f'{elementCD}']))
  
  return data

In [ ]:
# Function to Download Multiple Stations at a time from AWDB Web Service

# stations = ['401:OR:SNTL', '471:ID:SNTL', '591:WA:SNTL']
def getData(stations, parameter_of_interest, begindate, enddate):
  # data = []
  # data_singleDF
  # for i in stations:
  #   data.append(SOAP_Call(stationtriplets=i,elementCD=parameter_of_interest,begindate=begindate,enddate=enddate))

  # data_singleDF =
 
  data_singleDF = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [SOAP_Call(stationtriplets=j,elementCD=parameter_of_interest,begindate=begindate,enddate=enddate) for j in stations])
  # data_singleDF.rename(columns='bla')
  data_singleDF.columns = [f'{j}' for j in stations]

  # index = np.array([data_singleDF.columns.values, data_singleDF.iloc[1].values])
  # data_singleDF = pd.MultiIndex.from_arrays(index)
  # data_singleDF = data_singleDF.iloc[1:]

  # data_singleDF.xs('price', axis=1, level=1, drop_level=False)


  return data_singleDF

In [ ]:
def zScore_plot(df, max_StDev=2):
  global zs, df_outliers
  df2 = pd.DataFrame(df)
  zs = zscore(df2, ddof=2, nan_policy='omit')
  # zs = pd.DataFrame(zs)
  df2.insert(loc=0, column='zscore', value=zs)
  df_outliers = df2[(df2['zscore'] < -max_StDev) | (df2['zscore'] > max_StDev)]

  fig = go.Figure()
  fig.add_trace(go.Scatter(
    x=df.index,
    y=df,
    mode='lines',
    # name=station_of_interest,
    # hovertext = ORWA_Sites2[station_of_interest][parameter_of_interest],   
  ))
  fig.add_trace(go.Scatter(
    x=df_outliers.index,
    y=df_outliers.iloc[:,1],
    mode='markers',
    # name=station_of_interest,
    hovertext = df_outliers['zscore'],   
  ))
  fig.update_xaxes(
      title_text= 'Date'
  )
  # fig.update_yaxes(title_text= parameter_of_interest)
  fig.update_layout(
    height=800,
    width=1100,
  )

  fig.show()

# Creating logicalTests and outlierDetection Classes

In [ ]:
# class errorChecking():
#   def __init__(self, stationtriplets, begindate, enddate):
#     global df2   
#     self.stations = stationtriplets
#     self.begindate = begindate
#     self.enddate = enddate
#   """
#   """
class logicalTests():
  def __init__(self, stationtriplets, begindate, enddate):
    self.stations = stationtriplets
    self.begindate = begindate
    self.enddate = enddate
    
    global df, df2
    WTEQ = getData(self.stations, 'WTEQ', self.begindate, self.enddate)
    PREC = getData(self.stations, 'PREC', self.begindate, self.enddate)
    SNWD = getData(self.stations, 'SNWD', self.begindate, self.enddate)

    df = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [WTEQ, PREC, SNWD])
    # df = pd.merge(WTEQ, PREC, left_index=True, right_index=True).reset_index()
    df.reset_index(inplace=True)
    df.columns = ['Date', 'WTEQ (in)', 'PREC (in)', 'SNWD (in)']

    #Mask to select only data between Nov - Apr
    winter_filter = ~pd.to_datetime(df['Date']).dt.month.between(5,10)
    df2 = df.where(winter_filter).dropna()

    #Add column specifying the water year (Oct 1 - Sep 30) of each record
    df2.reset_index(inplace=True)  #Have to reset the index so the dates become just a regular datetime object to get .dt.year to work.
    pd.to_datetime(df2['Date'])
    df2['water_year'] = pd.to_datetime(df2['Date']).dt.year.where(pd.to_datetime(df2['Date']).dt.month < 10, pd.to_datetime(df2['Date']).dt.year + 1)
    df2['water_year'] = list(map(lambda x: str(x), df2['water_year']))

    self.df2 = df2
    self.WTEQ = WTEQ
    self.PREC = PREC

  def plots(self):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
      x=df2['Date'],
      y=df2['WTEQ (in)'],
      mode='lines',
      name='WTEQ (in)',
      hovertext = df2['WTEQ (in)']   
    ))
    fig.add_trace(go.Scatter(
      x=df2['Date'],
      y=df2['PREC (in)'],
      mode='lines',
      name='PREC (in)',
      hovertext = df2['PREC (in)'],   
    ))
    fig.add_trace(go.Scatter(
      x=df2['Date'],
      y=df2['SNWD (in)'],
      mode='lines',
      name='SNWD (in)',
      hovertext = df2['SNWD (in)'],   
    ))
    fig.update_xaxes(title_text= 'Date')
    # fig.update_yaxes(title_text= parameter_of_interest)
    fig.update_layout(
      height=800,
      width=1100,
    )
    fig.show() 
  
  def WTEQvsPREC(self):  
    """
    Plots WTEQ vs PREC on a fixed scale line plot
    """
    fig = go.Figure()

    fig = px.line(self.df2, x="PREC (in)", y="WTEQ (in)", hover_name=self.df2['Date'], color='water_year')
    fig.update_layout(
        yaxis=dict(scaleanchor="x", scaleratio=1),
        height=800,
        width=1100)

    fig.show()

  def SWEvsSNWD(self):  
    """
    Plots SWE vs SNWD on a fixed scale line plot
    """
    fig = go.Figure()

    fig = px.line(self.df2, x="WTEQ (in)", y="SNWD (in)", hover_name=self.df2['Date'], color='water_year')
    fig.update_layout(
        yaxis=dict(scaleanchor="x", scaleratio=1),
        height=800,
        width=1100)

    fig.show()
    
  def WTEQvsPREC_table(self, max_daily_WTEQ, max_daily_PREC):

  #Calculate Precip increment from accumulation:
    self.df2['Precip Increment - Calculated (in)'] = self.df2['PREC (in)'].diff()
    self.df2['WTEQ - Increment (in)'] = self.df2['WTEQ (in)'].diff()


  #Filter to retrieve potentially erroneous data
    df_errors = self.df2[(self.df2['Precip Increment - Calculated (in)'] <= max_daily_PREC) & (self.df2['WTEQ - Increment (in)'] >= max_daily_WTEQ)].sort_values('WTEQ - Increment (in)', ascending=False)
    # df_errors.to_excel('./WTEQvsPrec/'+f'{station}' + '_WTEQvsPrecipErrors.xlsx')
    df_errors.set_index('Date', inplace=True)
    df_errors.drop('index', axis=1, inplace=True)

    return df_errors

#   # def calc_SWEvsPREC_slope():

  def precipitationLessThanZero(self):
    # PREC = getData(self.stations, 'PREC', self.begindate, self.enddate)
    PrecipLessThanZero_Errors = self.PREC[self.PREC.iloc[:,0] < 0]
    return PrecipLessThanZero_Errors

In [ ]:
 class outlierDetection():
    def __init__(self, stationtriplets, begindate, enddate):
      self.stations = stationtriplets
      self.begindate = begindate
      self.enddate = enddate

    def temp_boxplots(self, data_pt_type):
      """
      options for data_pt_type include: 'all', 'outliers', or 'suspectedoutliers'
      """      
      TAVG = getData(self.stations, 'TAVG', self.begindate, self.enddate)
      TMAX = getData(self.stations, 'TMAX', self.begindate, self.enddate)
      TMIN = getData(self.stations, 'TMIN', self.begindate, self.enddate)
      TOBS = getData(self.stations, 'TOBS', self.begindate, self.enddate)

      df_temp = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [TAVG, TMAX, TMIN, TOBS])
      df_temp.columns = ['TAVG (degF)','TMAX (degF)','TMIN (degF)','TOBS (degF)']

      fig = go.Figure()
      for i in df_temp.columns:
      #   WA = WA_Sites[WA_Sites[i] != 0].dropna()

        fig.add_trace(go.Box(y = df_temp.loc[:,i], 
                name = i,
                boxpoints=data_pt_type, 
                # hovertext = str(df_temp[i].index),
                jitter=0.2,
                whiskerwidth=0.5                
      ))

      fig.update_layout(
              autosize=False,
              width=1200,
              height=500,
              margin=dict(l=20, r=20, t=2, b=8),
              paper_bgcolor="LightSteelBlue",
              title={
              # 'text': f'{station}',
              'y':0.98,
              'x':0.45,
              'xanchor': 'center',
              'yanchor': 'top'
      })
      fig.show()

    def zScore_plots(self):
      global zs, df_outliers
      for i in ['TAVG', 'TMIN', 'TMAX', 'TOBS']:
        df = getData(stations, i, self.begindate, self.enddate)
        zScore_plot(df.iloc[:,0])

## Test Runs

In [ ]:
OR302 = outlierDetection(['302:OR:SNTL'], '04/01/2018', '04/01/2020')

In [ ]:
OR302.zScore_plots()

In [ ]:
stations = ['302:OR:SNTL']
begindate  = '04/01/2018'
enddate  = '04/01/2020'

for i in ['TAVG', 'TMIN']:
  df = getData(stations, i, begindate, enddate)
  zScore_plot(df.iloc[:,0], max_StDev=2)

In [ ]:
df

,zscore,302:OR:SNTL
Date,,
2018-04-02,-0.388018,30.56
2018-04-03,-1.502843,16.16
2018-04-04,-0.792143,25.34
2018-04-05,-0.165053,33.44
2018-04-06,0.057912,36.32
...,...,...
2020-03-29,-0.485566,29.30
2020-03-30,-0.374083,30.74
2020-03-31,-0.499501,29.12


In [ ]:
stations = ['302:OR:SNTL']
begindate  = '04/01/2018'
enddate  = '04/01/2020'

df = getData(stations, 'TAVG', begindate, enddate)
# TMAX = getData(stations, 'TMAX', begindate, enddate)
# TMIN = getData(stations, 'TMIN', begindate, enddate)
# TOBS = getData(stations, 'TOBS', begindate, enddate)

df_temp = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [TAVG, TMAX, TMIN, TOBS])
# df_temp.columns = ['TAVG (degF)','TMAX (degF)','TMIN (degF)','TOBS (degF)']

def zScore_plot(df_temp, degrees_of_freedom):
 
  zs = zscore(df.iloc[:,0], ddof=2, nan_policy='omit')
  df_outliers = zs[(zs['zscore'] < -3) | (zs['zscore'] > 3)].head(45)

  fig = go.Figure()

  fig.add_trace(go.Scatter(
    x=ORWA_Sites2.index,
    y=ORWA_Sites2[station_of_interest][parameter_of_interest],
    mode='lines',
    name=station_of_interest,
    hovertext = ORWA_Sites2[station_of_interest][parameter_of_interest],   
  ))

  fig.add_trace(go.Scatter(
    x=df_outliers.index,
    y=df_outliers[station_of_interest][parameter_of_interest],
    mode='markers',
    name=station_of_interest,
    hovertext = df_outliers.index,   
  ))

  fig.update_xaxes(title_text= 'Date')
  fig.update_yaxes(title_text= parameter_of_interest)

  fig.update_layout(
    height=800,
    width=1100,
  )

  fig.show()

In [ ]:
asdf = outlierDetection(['302:OR:SNTL'],'01/01/2018', '04/01/2020')

In [ ]:
df_temp

NameError: ignored

In [ ]:
asdf.temp_boxplots('outliers')

In [ ]:
bla = logicalTests(['401:OR:SNTL'],'04/01/2021', '04/13/2021')

In [ ]:
bla.plots()

In [ ]:
bla.precipitationLessThanZero()

,401:OR:SNTL
Date,


In [ ]:
bla.WTEQvsPREC_table(0.5,0.2)

,WTEQ (in),PREC (in),water_year,Precip Increment - Calculated (in),WTEQ - Increment (in)
Date,,,,,


In [ ]:
bla.WTEQvsPREC()

In [ ]:
bla.SWEvsSNWD()

In [ ]:
OR401 = errorChecking(['401:OR:SNTL'],'01/01/2018', '04/01/2020')

In [ ]:
OR401.logicalTests.WTEQvsPREC()

NameError: ignored

In [ ]:
OR401.logicalTests.WTEQvsPREC_table(0.2,0.5)

,WTEQ (in),PREC (in),water_year,Precip Increment - Calculated (in),WTEQ - Increment (in)
Date,,,,,
2020-01-12,1.5,10.5,2020,0.4,0.9
2018-02-27,3.6,23.8,2018,0.3,0.8
2019-03-14,11.8,26.0,2019,0.4,0.7
2018-03-25,5.9,27.4,2018,0.5,0.6
2019-02-14,6.7,20.8,2019,0.4,0.6
...,...,...,...,...,...
2018-02-18,0.9,21.7,2018,0.1,0.2
2018-01-23,0.9,18.2,2018,0.1,0.2
2018-02-19,1.1,22.0,2018,0.3,0.2


In [ ]:
OR401.logicalTests.precipitationLessThanZero()

,401:OR:SNTL
Date,


In [ ]:
OR401.WTEQvsPREC_table(0.3,0.2)

,WTEQ (in),PREC (in),water_year,Precip Increment - Calculated (in),WTEQ - Increment (in)
Date,,,,,
2014-12-29,0.8,18.9,2015,0.2,0.6
2020-02-17,5.0,20.0,2020,0.2,0.5
2014-02-22,7.0,20.7,2014,0.2,0.5
2018-04-14,4.9,30.2,2018,0.2,0.5
2014-02-21,6.5,20.5,2014,0.2,0.4
2017-03-02,15.3,32.3,2017,0.2,0.4
2015-01-06,2.7,20.4,2015,0.2,0.4
2017-11-19,1.3,8.8,2018,0.1,0.4
2019-02-18,7.6,21.6,2019,0.1,0.4
